# Fake Cross-section problem for TG Testing

In [1]:
import material_pb2 as mat
import numpy as np

Energy groups and $\chi$

In [2]:
energy_groups = [2e7, 1.353e6, 9.119e3, 3.928, 0.6251, 0.1457, 0.0569, 0]
normalized_chi = [0.5, 0.25, 0.25, 0, 0, 0, 0]

Base location to save files:

In [3]:
folder = "/home/josh/repos/bart/benchmarks/tgnda_fake_cross_sections/"

In [4]:
def make_material(name, sigma_t, sigma_s, is_fissionable, nu_sig_f):
    material = mat.Material()
    material.full_name = name
    material.id = name
    material.abbreviation = name
    material.is_fissionable = is_fissionable
    material.number_of_groups = 7

    eg = mat.Material.VectorProperty()
    eg.id = mat.Material.ENERGY_GROUPS
    eg.value.extend(energy_groups)

    material_sigma_t = mat.Material.VectorProperty()
    material_sigma_t.id = mat.Material.SIGMA_T
    material_sigma_t.value.extend(sigma_t)

    diff = mat.Material.VectorProperty()
    diff.id = mat.Material.DIFFUSION_COEFF
    diff.value.extend([1.0/(3.0 * val) for val in sigma_t])

    if is_fissionable:
        nsf = mat.Material.VectorProperty()
        nsf.id = mat.Material.NU_SIG_F
        nsf.value.extend(nu_sig_f)

        chi = mat.Material.VectorProperty()
        chi.id = mat.Material.CHI
        chi.value.extend(normalized_chi)

    material_sigma_s = mat.Material.MatrixProperty()
    material_sigma_s.id = mat.Material.SIGMA_S
    material_sigma_s.value.extend(sigma_s)

    if is_fissionable:
        material.vector_property.extend([eg, material_sigma_t, nsf, chi, diff])
    else:
        material.vector_property.extend([eg, material_sigma_t, diff])
    material.matrix_property.extend([material_sigma_s])

    filename = folder + name + ".material"
    f = open(filename, 'wb')
    f.write(material.SerializeToString())
    f.close()

----
## Materials

### High scattering fissionable material

In [8]:
name = "high_scattering_fissionable"
sigma_t = [1, 1, 1, 1, 1, 1, 1]

sigma_s = np.array([[0.499, 0,    0,    0,    0,    0,    0],
                    [0.25, 0.499, 0,    0,    0,    0,    0],
                    [0.25, 0.499, 0.459, 0,    0,    0,    0],
                    [0,    0,    0.24, 0.3,  0.29, 0.25,  0.2],
                    [0,    0,    0.1,  0.23,  0.3,  0.24,  0.2],
                    [0,    0,    0.1,  0.23,  0.2,  0.3 , 0.2],
                    [0,    0,    0.1,  0.23,  0.2,  0.2,  0.3]])

nu = 2
nu_sig_f = []

for group in range(7):
    sigma_s[:,group] *= sigma_t[group] * 0.5
    sigma_a = sigma_t[group] - np.sum(sigma_s[:,group])
    nu_sig_f.append(nu * 0.95*sigma_a)

sigma_s_list = []
for row in sigma_s:
    for val in row:
        sigma_s_list.append(val)
    
make_material(name, sigma_t, sigma_s_list, True, nu_sig_f)

### Non-fissionable reflector

In [6]:
name = "high_scattering_reflector"
sigma_t = [1, 1, 1, 1, 1, 1, 1]

sigma_s = np.array([[0.499, 0,    0,    0,    0,    0,    0],
                    [0.25, 0.499, 0,    0,    0,    0,    0],
                    [0.25, 0.499, 0.459, 0,    0,    0,    0],
                    [0,    0,    0.24, 0.3,  0.29, 0.25,  0.23],
                    [0,    0,    0.1,  0.23,  0.3,  0.24,  0.23],
                    [0,    0,    0.1,  0.23,  0.2,  0.3 , 0.23],
                    [0,    0,    0.1,  0.23,  0.2,  0.2,  0.3]])

for group in range(7):
    sigma_s[:,group] *= sigma_t[group]
    sigma_a = sigma_t[group] - np.sum(sigma_s[:,group])

sigma_s_list = []
for row in sigma_s:
    for val in row:
        sigma_s_list.append(val)
    
make_material(name, sigma_t, sigma_s_list, False, [])